In [1]:
#https://github.com/uvipen/Character-level-cnn-pytorch

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import sys
import csv
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import shutil
import unicodedata
import string
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
pd.read_csv('dataset-cnn/cleaned/training.csv', header=None, sep=';')

,0,1
0,1,redis.clients.jedis.JedisPoolConfig.setNumTest...
1,1,org.elasticsearch.common.settings.Settings.set...
2,1,org.apache.catalina.deploy.FilterMap.addURLPat...
3,1,org.apache.sshd.server.sftp.SftpSubsystem.proc...
4,1,org.mockito.Matchers.notNull()__XX__org.mockit...
...,...,...
1168,0,org.sonarqube.ws.client.qualitygates.Qualityga...
1169,0,org.sonarqube.ws.client.qualitygates.Qualityga...
1170,0,org.sonarqube.ws.client.qualitygates.Qualityga...
1171,0,org.sonarqube.ws.client.qualitygates.ProjectSt...


In [4]:
class MyDataset(Dataset):
    def __init__(self, file, maxl=200, transform=None, target_transform=None):
        self.vocabulary = list("""abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}""")
        self.identity_mat = np.identity(len(self.vocabulary))
        self.arr = pd.read_csv(file, sep=';', header=None)
        self.transform = transform
        self.target_transform = target_transform
        self.max_length = maxl

    def __len__(self):
        return len(self.arr)

    def __getitem__(self, idx):
        label = self.arr.iloc[idx, 0]
        line = self.arr.iloc[idx, 1]
        
        line = np.array([self.identity_mat[self.vocabulary.index(i)] for i in list(line) if i in self.vocabulary],
                        dtype=np.float32)
        
        if len(line) > self.max_length:
            line = line[:self.max_length]
        elif 0 < len(line) < self.max_length:
            line = np.concatenate(
                (line, np.zeros((self.max_length - len(line), len(self.vocabulary)), dtype=np.float32)))
        elif len(line) == 0:
            line = np.zeros((self.max_length, len(self.vocabulary)), dtype=np.float32)
            
    
        return label, line
    

In [5]:
training_dataset = MyDataset('dataset-cnn/cleaned/training.csv')
validation_dataset = MyDataset('dataset-cnn/cleaned/validation.csv')

In [6]:
training_dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=64, shuffle=True)

In [7]:
dataiter = iter(training_dataloader)
labels, lines = dataiter.next()

print(type(labels))
print(type(lines))

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [8]:
print(labels)

tensor([1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1])


In [9]:
print(lines[1][0])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [15]:
class CharacterLevelCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(200, 200, 15)
#         self.pool = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(200, 200, 15)
        self.fc1 = nn.Linear(2400, 1200)
        self.fc2 = nn.Linear(1200, 840)
        self.fc3 = nn.Linear(840, 2)
    
    def forward(self, x):
#         print(x)
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
model = CharacterLevelCNN()

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

In [19]:
for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(training_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        labels, lines = data
#         print(data)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(lines)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')

print('Finished Training')

[1,     1] loss: 0.014
[2,     1] loss: 0.014
[3,     1] loss: 0.014
[4,     1] loss: 0.014
[5,     1] loss: 0.014
[6,     1] loss: 0.014
[7,     1] loss: 0.014
[8,     1] loss: 0.014
[9,     1] loss: 0.014
[10,     1] loss: 0.014
[11,     1] loss: 0.014
[12,     1] loss: 0.014
[13,     1] loss: 0.014
[14,     1] loss: 0.014
[15,     1] loss: 0.014
Finished Training


In [ ]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 500

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()